In [2]:
ls ./og_model_genome_files

extracted_genes.fasta  genomic.fasta.fai  results.txt
genomic.fasta          genomic.gff


In [51]:
# convert blast_id to ZM id for model genes
f = open('./og_model_genome_files/genomic.gff', 'r')
lines = f.readlines()
f.close()
id_to_locus = {}
for line in lines[7:-1]:
    opts = line.split('\t')
    if opts[2] != 'gene':
        continue
    new_name = opts[0]+':'+str(int(opts[3])-1)+'-'+opts[4]
    id_to_locus.update({new_name : opts[-1].split('Name=')[1].split(';')[0]})


In [52]:
# convert model genes to biocyc genes
f = open('./og_model_genome_files/results.txt', 'r')
lines = f.readlines()
f.close()
ZM_gene_to_biocyc = {}
for line in lines:
    opts = line.split('\t')
    if opts[0] not in id_to_locus:
        continue
    ZM_gene = id_to_locus[opts[0]]
    biocyc_gene = opts[1].split('|')[2]
    ZM_gene_to_biocyc.update({ZM_gene : biocyc_gene})

In [83]:
# convert model values
f = open('./inputs/i_ZM4_481.json', 'r')
lines = f.readlines()
f.close()
f = open('./inputs/converted_model.json', 'w')
new_lines = []
for line in lines:
    if 'kegg.gene' in line:
        pre_gene = line.split('zmo')[0]
        gene = line.split('zmo:')[1].split('\"')[0]
        if gene in ZM_gene_to_biocyc:
            new_gene = ZM_gene_to_biocyc[gene]
        else:
            new_gene = gene
        post_gene = '\"'+post_gene.split('\"')[1]
        new_line = pre_gene+new_gene+post_gene
    else:
        new_line = line
    f.write(new_line)
f.close()